# STYLE

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/style_transfer.ipynb


https://github.com/manicman1999/Style-Transfer-TF2.0



In [0]:
REGET = 0 # get again data
GDRIVE = 1 # mount gdrive: gdata, gwork
ING = 1 # ckpt in gwork
CLEARTMP = 0 # clear g tmp folder 
MODITEM = "" # will look into module

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x 
except Exception:
    pass

import tensorflow as tf
from tensorflow.keras import layers

tf.__version__

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
import matplotlib.image as mpimg

import numpy as np
import PIL
import PIL.Image

import time
import functools
import random

import os

import zipfile

from IPython import display

import matplotlib.pyplot as plt

# To generate GIFs
!pip install -q imageio
import glob
import imageio


# GDATA

In [0]:
# mount gdrive and set gdata
import os

# gdrive
try:
    GDRIVE
except NameError:
    GDRIVE = 1 # if not exist assume want to mount gdrive

if GDRIVE:
    from google.colab import drive # drive from colab
    drive.mount('/content/gdrive', force_remount=True)

gdrive = '/content/gdrive/My Drive'
if not os.path.exists(gdrive):        
    print("NO gdrive")

# gdata
gdata = ''
if os.path.exists(gdrive):        
    gdata = '/content/gdrive/My Drive/gdata/' 
    print ('gdata is %s\n' %gdata)

    if os.path.exists(gdata):        
        !ln -s "$gdata" /content/gdata
else:
    gdata = '/content/gdata/' 
    print ('gdata is %s\n' %gdata)

if not os.path.exists(gdata):
    os.mkdir(gdata)

# gwork
gwork = ''
if os.path.exists(gdrive):        
    gwork = '/content/gdrive/My Drive/gwork/' 
    print ('gwork is %s\n' %gwork)

    if os.path.exists(gwork):        
        !ln -s "$gwork" /content/gwork
else:
    gwork = '/content/gwork/' 

if not os.path.exists(gwork):
    os.mkdir(gwork)
print ('gwork is %s\n' %gwork)


In [0]:
# clear work area
try:
    CLEARTMP
except NameError:
    CLEARTMP = 0 # if not exist assume not to clear

if CLEARTMP:
    for root, dirs, files in os.walk(gwork, topdown=False):

        print("clear %s" %root)
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))


# IMAGINE

In [0]:
# IMAGINE 

import matplotlib.pyplot as plt
import IPython.display as display

class IMAGINE(object):

    def __init__(self,
                 verbose = False):
        self.verbose = verbose

    # ******************
    #  tensor_to_image
    #
    def tensor_to_image(self, tensor):
        tensor = tensor*255
        tensor = np.array(tensor, dtype=np.uint8)
        if np.ndim(tensor)>3:
            assert tensor.shape[0] == 1
            tensor = tensor[0]
        return PIL.Image.fromarray(tensor)

    # ******************
    #  imshow
    #  tf.squeeze: removes dimensions of size 1 from the shape of a tensor
    #
    def imshow(self, image, title=None):
        if len(image.shape) > 3:
            image = tf.squeeze(image, axis=0)

        plt.imshow(image)
        if title:
            plt.title(title)    

    # ******************
    #  high_pass_x_y
    #   high frequency components of the image
    #   ref: https://www.tensorflow.org/tutorials/generative/style_transfer
    #
    def high_pass_x_y(self, image):
        x_var = image[:,:,1:,:] - image[:,:,:-1,:]
        y_var = image[:,1:,:,:] - image[:,:-1,:,:]

        return x_var, y_var

    # ******************
    #  clip_0_1
    #  function to keep the pixel values between 0 and 1
    #
    def clip_0_1(self, image):

        return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)                

    # ******************
    #  random_crop
    #
    def random_crop(self, img, w=256, h=256):

        cropped_image = tf.image.random_crop(
            img, size=[w, h, 3])

        return cropped_image

    # ******************
    #  normalize
    #
    # normalizing the images to [-1, 1]
    #
    def normalize(self, img):
        img = tf.cast(img, tf.float32)
        img = (img / 127.5) - 1

        return img

    # ******************
    #  preprocess_image
    #
    def preprocess_image(self, img, w=256, h=256):

        img = self.normalize(img)
        img = tf.image.resize(img, [w, h])

        return img

    # ******************
    #  preprocess
    #
    def preprocess(self, images, w=256, h=256, buffer_size=128):

        images = images.map(
            self.preprocess_image,
            num_parallel_calls=tf.data.experimental.AUTOTUNED
        ).cache().shuffle(buffer_size).batch(1)

        return images


    # ******************
    #  load_img
    #
    def load_img(self, path_to_img, max_dim = 512):
        
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)

        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img


    # ******************
    #   get_images
    #
    #   get image files from folder
    #
    def get_images(self, directory='dataset/'): # get images from local folder
        jpgs = glob.glob( '{}*.jpg'.format(directory) )
        pngs = glob.glob( '{}*.png'.format(directory) )
        allimages = jpgs + pngs
        return allimages

    # ******************
    #   load
    #
    def load(self, image_file):
        image = tf.io.read_file(image_file)
        image = tf.image.decode_jpeg(image)

        w = tf.shape(image)[1]

        w = w // 2
        real_image = image[:, :w, :]
        input_image = image[:, w:, :]

        input_image = tf.cast(input_image, tf.float32)
        real_image = tf.cast(real_image, tf.float32)

        return input_image, real_image

    # ******************
    #   random_jitter
    #
    def random_jitter(self, image):
        # resizing to 286 x 286 x 3
        image = tf.image.resize(image, [286, 286],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        # randomly cropping to 256 x 256 x 3
        image = random_crop(image)

        # random mirroring
        image = tf.image.random_flip_left_right(image)

        return image

    # ******************
    #  tiles
    #
    def tiles(self, images, rows = 4, cols = 4):
        for i in range(len(images)):
            plt.subplot(images, rows, i+1)
            plt.title(i)
            if i % cols == 0:
                plt.imshow(images[i][0] * 0.5 + 0.5)
            else:
                plt.imshow(images[i][0] * 0.5 * contrast + 0.5)
        plt.show()

    # ******************
    #  create_image
    #
    def create_image(self, images, name='create_img_name', xsize=4, ysize=4):
        
        fig, axs = plt.subplots(xsize, ysize, figsize=(xsize*2,ysize*2))
        plt.subplots_adjust(left=0.05,bottom=0.05,right=0.95,top=0.95, wspace=0.2, hspace=0.2)

        cnt = 0
        for i in range(ysize):
            for j in range(xsize):
                axs[i,j].imshow(images[cnt])
                axs[i,j].axis('off')
                cnt += 1

        fig.savefig(name, facecolor='white' )      
        plt.close()

    # ******************
    #  genimages tbd
    #
    def genimages(self, images, cols = 2):
        sets =[images[i:i + cols] for i in range(0, len(images), cols)]
        for s in sets:
            for i in range(cols):
                plt.subplot(1, cols, i+1)
                plt.title(i)
                plt.imshow(images[cols * s + i])
                plt.axis('off')
            plt.show()


    

# ANIME

In [0]:
# ANIME

import argparse
import time
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
 
from skimage.transform import rotate 
from skimage.color import rgb2hsv, hsv2rgb

from moviepy.editor import *
from moviepy.Clip import Clip
from IPython.display import display


from moviepy.video.VideoClip import VideoClip
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.video.io.ffmpeg_reader import FFMPEG_VideoReader


class ANIME():
    def __init__(self, 
                 xSize=128, # dataset
                 ySize=128, # dataset
                 nSlices=100, # dataset
                 resize=1, # dataset
                 directory='gainsborough/', # dataset
                 name= 'gainsborough_256_128', # dataset
                 latent_dim=256, # model
                 range_plot=4, # plot
                 epochs=101, # train
                 batch_size=32, # train
                 save_interval=50, # train
                 channels=3,
                 imgXsize = 512, # image
                 imgYsize = 512, # image
                 img_rows = 128, # image
                 img_cols = 128, # image
                 rows = 4, # mosaic
                 columns = 4, # mosaic

                 fps = 30, # video settings
                 maxTime = 5, # 30 # seconds # video settings
                 frameCount = 0, # video settings
                 time = 0, # video settings

                 ):
        
        # self.fig, self.axs = plt.subplots(rows,columns, figsize=(4,4), facecolor=(1,1,1) )
        # plt.subplots_adjust(left=0.05,bottom=0.05,right=0.95,top=0.95, wspace=0.2, hspace=0.2)

        # self.fig.set_size_inches(size[0] / 100, size[1] / 100)
        self.images = []
        self.size=(imgXsize, imgYsize)
        self.ntales = rows * columns # tales in frame
        self.maxTime = maxTime

        self.nframes = int( maxTime*fps ), # video settings

    #   *******************
    #   add
    #
    #   add images to self.images
    #
    def add(self, images):
        imgs = []
        for k in range(16):
            ax = self.axs[int(k/4.),k%4].imshow(images[k])
            self.axs[int(k/4.),k%4].axis('off')
            imgs.append(ax)
        self.images.append(imgs)

    #   *******************
    #   save
    #
    #   save animation
    #
    def save(self, filename, fps=1):
        animation = anim.ArtistAnimation(self.fig, self.images)
        animation.save(filename, fps=fps,
            #progress_callback = lambda i, n: print(f'Saving frame {i} of {n}')
        )

    #   *******************
    #   anim
    #
    #   display.Image(filename=anim_file)
    #
    def anim(self, srcfolder, anim_file):
        with imageio.get_writer(anim_file, mode='I') as writer:
            filenames = glob.glob(srcfolder + '*.jpg')
            filenames = sorted(filenames)
            last = -1

            # ingsn = len(filenames) / 2
            for i in range(len(filenames)):
                plt.subplot(2, 2, i+1)
                print(filenames[i])
                image = Imagine.load_img(filenames[i])
                plt.imshow(image[0])
                plt.show()

            for i,filename in enumerate(filenames):
                print(filename)
                img = Imagine.load_img(filename)
                Imagine.imshow(img)
                frame = 2*(i**0.5)
                if round(frame) > round(last):
                    last = frame
                else:
                    continue
                image = imageio.imread(filename)

            writer.append_data(image)
            image = imageio.imread(filename)
            writer.append_data(image)

    #   *******************
    #   make_fab
    #
    def make_fab(self, model):
        
        # model.load_weights(generator_file="generator ({}).h5".format(model.name), \
        #                    discriminator_file="discriminator ({}).h5".format(model.name))
        model.load_weights(generator_file="{}.tf".format(model.name),
                           discriminator_file="{}.tf".format(model.name))
        # # video settings
        # fps = fps
        # maxTime = maxTime # seconds
        # frameCount = frameCount
        # time = time
        # nframes = nframes

        # controls for animation
        seed_start = np.random.normal(1, 1, (16, model.latent_dim))
        latentSpeed = np.random.normal(2, 1, (16, model.latent_dim))
        vary = np.random.normal(1, 1, (16, nframes, model.latent_dim)) 

        # randomize image transformations
        #rhue =  np.random.random()
        #rotation = 360 * np.round(np.random.random((4,))*4)/4 # random rotation
        #flip = np.random.randint(0,4,(4,)) # 0=normal, 1=y-axis, 2=x-axis, 3=transpose 

        # latent parameter animation
        for k in range(16):
            time = 0

            # for each image in animation 
            for i in range(nframes): 
                
                # change the latent variables
                for j in range(model.latent_dim):
                    vary[k][i][j] = seed_start[k][j] + np.sin( 2*np.pi*(time/maxTime) * latentSpeed[k][j] ) 

                time += 1./fps

        imgs = []
        for k in range(16):
            imgs.append(
                model.generator.predict(vary[k])
            )
        imgs = np.array(imgs)

        # create animation
        animated_gif = AnimatedGif()
        for i in range(nframes):
            
            animated_gif.add(imgs[:,i])

        animated_gif.save('artificial_art.mp4',fps=fps)

        # dude() 

        count = np.loadtxt('count.txt')

        with open('hashtags.txt') as fp:
            hashtags = fp.readlines()
        hashtags = [hashtags[i].strip() for i in range(len(hashtags))]

        message = "Automated Artificial Art v 1.{:.1f} - machine hallucinations from an artificial neural network \n \n#".format(count[0])
        message = message + " #".join( np.random.choice(hashtags,4))


    # Display a single image using the epoch number

    def display_image(self, epoch_no):
        return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

    def display_lastimage(self):
        trainer = self.trainer
        epoch_no = trainer.epochs
        return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

    #   ************
    #   generate_gif
    #
    def generate_gif(self, 
                     srcfolder = None,
                     dstfile = 'ani.gif',
                     dstfolder = None,
                     ):


        print("srcfolder %s" %srcfolder)
        with imageio.get_writer(dstfile, mode='I') as writer:
            if srcfolder:
                imgfiles = srcfolder + '/' + '*.png'
            else:
                imgfiles = '*.png'

            filenames = glob.glob(imgfiles)
            filenames = sorted(filenames)

            print("filenames %s" %filenames)
            last = -1
            for i,filename in enumerate(filenames):
                frame = 2*(i**0.5)
                if round(frame) > round(last):
                    last = frame
                else:
                    continue
                image = imageio.imread(filename)
                writer.append_data(image)

        import IPython
        if IPython.version_info >= (6,2,0,''):
            display.Image(filename=dstfile)

    #   ************
    #   display_gif
    #
    def display_gif(self, giffile = 'ani.gif', height=256, autoplay=0, loop=1):

        vids = [giffile]
        for i, vid_name in enumerate(vids):
            vid_path = '%s' %vid_name
            print ("vid_path %s" %vid_path)
            clip = VideoFileClip(vid_path)
            display(clip.ipython_display(height, autoplay, loop))    

    #   ************
    #   display_history
    #
    def display_history(self):

        acc = history.history['acc']
        val_acc = history.history['val_acc']
        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs = range(len(acc))

        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.legend(loc=0)
        plt.figure()


        plt.show()

  

# EXCHECKER

In [0]:
#   EXCHECKER
#
#   Manage model state
#
try:
    gwork
except NameError:|
    gwork = ''

class EXCHECKER(tf.keras.models.Model):
    def __init__(self,
                 model,
                 ckptpath = 'training_checkpoints',
                 clear = True
                 ):
        
        super(EXCHECKER, self).__init__()
        
        self.clear = clear
        self.model = model
        self.ckptpath = ckptpath
        self.step = 1

    #   ******************
    #   ckpt_sample
    #
    #
    def ckpt_sample(self):
        ckpt = tf.train.Checkpoint(v=tf.Variable(0.))
        mgr = tf.train.CheckpointManager(ckpt, '/tmp/tfckpts', max_to_keep=5)

        @tf.function
        def train_fn():
            data = tf.data.Dataset.range(10)
            for item in data:
                ckpt.v.assign_add(tf.cast(item, tf.float32))
                tf.py_function(mgr.save, [], [tf.string])

        train_fn()


    #   ******************
    #   get_ckpt_prefix
    #
    #
    def get_ckpt_prefix(self):
        ckptpath = self.ckptpath

        return ckptpath

    #   ******************
    #   get_checkpoint
    #
    #
    def get_checkpoint(self):
        print("exchecker.get_checkpoint")
        model = self.model
        p = model.chkpt

        ckpt = tf.train.Checkpoint(**p)
        return ckpt

    #   ******************
    #   get_manager
    #
    #
    def get_manager(self, ckpt):
        model = self.model
        ckptpath = model.ckptpath

        if self.clear:
            if os.path.exists(ckptpath):
                for root, dirs, files in os.walk(ckptpath, topdown=False):
                    print("clear ckpt dir: %s" %root)
                    for name in files:
                        os.remove(os.path.join(root, name))
                    for name in dirs:
                        os.rmdir(os.path.join(root, name))
                
        print("exchecker get_manager")
        max_to_keep = 3
        manager = tf.train.CheckpointManager(ckpt, 
                                             ckptpath, 
                                             max_to_keep)
        print("exchecker got manager")        
        return manager

    #   ******************
    #   restore
    #
    #   if a checkpoint exists, restore the latest checkpoint
    #   eg:
    #       ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
    #       if ckpt_manager.latest_checkpoint:
    #           ckpt.restore(ckpt_manager.latest_checkpoint)
    #           print ('Latest checkpoint restored!!')
    #
    def restore(self, ckpt):
        model = self.model
        ckptpath = self.ckptpath
         
        print("checkpoint restore %s" %ckpt)
        res = ckpt.restore(tf.train.latest_checkpoint(ckptpath))
        print("res: in % s checkpoint: %s" %(ckptpath, res))


    # checkpoint_path = "./checkpoints/train"
    # ckpt = tf.train.Checkpoint(generator_g=generator_g,
    #                            generator_f=generator_f,
    #                            discriminator_x=discriminator_x,
    #                            discriminator_y=discriminator_y,
    #                            generator_g_optimizer=generator_g_optimizer,
    #                            generator_f_optimizer=generator_f_optimizer,
    #                            discriminator_x_optimizer=discriminator_x_optimizer,
    #                            discriminator_y_optimizer=discriminator_y_optimizer)





# DATAGEN

In [0]:
#   DATAGEN

import numpy as np
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.util.tf_export import keras_export

from shutil import copyfile
from shutil import copy

class DATAGEN():
    def __init__(self, 
                 name='sol',
                 buffer_size = 60000,
                 batch_size = 256):


        self.name = name
        self.buffer_size = buffer_size
        self.batch_size = batch_size

    #   ******************
    #   clear_folder
    #
    #
    def clear_folder(self, dir):

        if os.path.exists(dir):
            for root, dirs, files in os.walk(dir, topdown=False):
                print("clear %s" %root)
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
                
    # ******************
    #  get_uriitems
    #
    #  get remote data 
    #   urd:    remote url srcfolder
    #   urns:   remote url src files
    #   gdir:   persistent folder
    #   wdir:   work folder
    #
    def get_uriitems(self, urd, urls, gdir, wdir, verbose = False):

        if 1: # if g src eq g raw, download to raw
            if not os.path.exists(wdir):
                if verbose:
                    print("create wdir: %s" %wdir)
                os.mkdir(wdir)
        
            if not os.path.exists(gdir):
                os.mkdir(gdir)
                if verbose:
                    print("create gdir %s and download urls" %gdir)

                for img in urls:
                    uri = urd + img
                    !wget --no-check-certificate $uri -P  $gdir

            if os.path.exists(gdir):
                for file in urls:
                    dstfile = os.path.join(wdir, file)
                    srcfile = os.path.join(gdir, file)

                    if os.path.exists(srcfile):
                        if verbose:
                            print("file %s in gdir" %(srcfile))
                        if not os.path.exists(dstfile):
                            if verbose:
                                print("copy cp %s to %s" %(srcfile, dstfile))
                            copy(srcfile, dstfile)

                    else:
                        print("file %s not available in gdir" %srcfile)
                        uri = urd + file
                        if verbose:
                            print("download from uri %s" %file)
                        !wget --no-check-certificate $uri -P  "$gdir"
                        if verbose:
                            print("copy %s to %s" %(srcfile, dstfile))
                        copy(srcfile, dstfile)


    # ******************
    #  get_rawdata
    #
    #  get remote data 
    #   urd:    remote url srcfolder
    #   urns:   remote url src files
    #   path:   local destination path
    #
    def get_rawdata(self, path, urd, urns):
        directory = path

        if not os.path.exists(directory):
            print("create imgfolder: %s" %directory)
            os.mkdir(directory)

        for img in urns:
            print(img)
            uri = urd + img
            !wget --no-check-certificate $uri -P  $directory


    # ******************
    #  get_gdata
    #
    #  get data from gdata 
    #   gpath:  gdata folder
    #   wpath:  work folder
    #
    def get_gdata(self, gpath, wpath, fnames):
        if os.path.exists(gpath):
            for file in fnames:
                src = os.path.join(gpath, file)
                dst = os.path.join(wpath, file)
                dstdir = wpath
                # https://docs.python.org/2/library/shutil.html
                print("cp %s to %s" %(src, dstdir))
                copy(src, dstdir)
        else:
            print("gsrc does not exist")        


    # ******************
    #  get_gfolder
    #
    #  get data from gdata 
    #   gpath:  gdata folder
    #   wpath:  work folder
    #
    def get_gfolder(self, gpath, wpath):
        if os.path.exists(gpath):
            for base, dirs, files in os.walk(gpath):
                for file in files:
                    src = os.path.join(base, file)
                    dst = os.path.join(wpath, file)
                    dstdir = wpath
                    # https://docs.python.org/2/library/shutil.html
                    print("cp %s to %s" %(src, dstdir))
                    copy(src, dstdir)
        else:
            print("gsrc does not exist")        




    # ******************
    #   get_images
    #
    #   get images from local folder
    #
    def get_images(self, path): 
        directory = path
        jpgs = glob.glob( '{}*.jpg'.format(directory) )
        pngs = glob.glob( '{}*.png'.format(directory) )
        allimages = jpgs + pngs
        return allimages

    #   ******************
    #   get_dataset
    #
    def get_dataset(self, 
                    path = "",
                    buffer_size = 60000,
                    batch_size = -1,                    
                    xSize=128, 
                    ySize=128, 
                    nSlices=100, 
                    resize=0.75, 
                    channels=3):
        
        allimages = self.get_images(path)

        x_train = []
        y_train = []

        nimages = len(allimages)
        print("n images: %s" %nimages)

        for i in range(len(allimages)):

            # load image
            img = Image.open(allimages[i])

            if resize != 1:
                # resizes image in-place
                img.thumbnail((img.size[0]*resize, img.size[1]*resize), Image.LANCZOS) 

            img_data = np.array(list(img.getdata())).reshape( (img.size[1],img.size[0],-1) ) 
            
            for n in range(nSlices):

                # create random slices 
                rx = np.random.randint( img.size[0]-xSize)
                ry = np.random.randint( img.size[1]-ySize)
            
                # pull out portion of ccd
                sub = np.copy(img_data[ry:ry+ySize, rx:rx+xSize]).astype(float)

                #x_train.append( preprocessing.scale(sub) )
                x_train.append( sub[:,:,:3]  ) 
                y_train.append( [rx,ry] )

        x_train = np.array(x_train)
        y_train = np.array(y_train)

        # return (x_train, y_train)

        print("x_train.shape: ", x_train.shape)

        x_train /= 255
        sliced_images =  tf.data.Dataset.from_tensor_slices(x_train)
        shuffled_images = sliced_images.shuffle(buffer_size)
        if batch_size > 0:
            batched_images = shuffled_images.batch(batch_size)
        else:
            batched_images = shuffled_images
        train_dataset = batched_images

        return train_dataset



    # ******************
    #   load_data
    #       'untar': False,
    #       'md5_hash': None,
    #       'file_hash': None,
    #       'cache_subdir': 'datasets',
    #       'hash_algorithm': 'auto',
    #       'extract': False,
    #       'archive_format': 'auto',
    #       'cache_dir': None       
    #
    def load_data(self, path='mnist.npz', urd=None, urf=None):

      path = get_file(
          path,
          origin=urd + urf)
      
      with np.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']

        return (x_train, y_train), (x_test, y_test) 


    # ******************
    #   download_file
    # 
    def download_file(self, **kwargs):
        # Finally, save the result

        file_name = 'stylized-image.png'
        Imagine.tensor_to_image(image).save(file_name)

        try:
            from google.colab import files
        except ImportError:
            pass
        else:
            files.download(file_name)

    # ******************
    #   create_dataset_np
    # 
    def create_dataset_np(self, xSize=128, ySize=128, nSlices=100, resize=0.75, directory='dataset/'):
        jpgs = glob.glob( '{}*.jpg'.format(directory) )
        pngs = glob.glob( '{}*.png'.format(directory) )

        allimages = jpgs + pngs

        x_train = []
        y_train = []

        for i in range(len(allimages)):

            # load image
            img = Image.open(allimages[i])

            if resize != 1:
                img.thumbnail((img.size[0]*resize, img.size[1]*resize), Image.LANCZOS) # resizes image in-place

            img_data = np.array(list(img.getdata())).reshape( (img.size[1],img.size[0],-1) ) 

            for n in range(nSlices):

                # create random slices 
                rx = np.random.randint( img.size[0]-xSize)
                ry = np.random.randint( img.size[1]-ySize)
            
                # pull out portion of ccd
                sub = np.copy(img_data[ry:ry+ySize, rx:rx+xSize]).astype(float)

                #x_train.append( preprocessing.scale(sub) )
                x_train.append( sub[:,:,:3]  ) 
                y_train.append( [rx,ry] )

        x_train = np.array(x_train)
        y_train = np.array(y_train)
        
        x_train /= 255 

        return (x_train, y_train)


# === MATCHUE

In [0]:
!ls -la "/content/gdrive/My Drive/gwork/flower_photos.tar.gz"

In [0]:
if 1:
    #Import packages
    import numpy as np
    import matplotlib.pyplot as plt

    from tensorflow.keras.layers import *
    from tensorflow.keras.models import *
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.optimizers import RMSprop

    #   MATCHUE
    #
    class MATCHUE(tf.keras.models.Model):

        def __init__(self,
                    image = None,
                    verbose = True,
                    ):

            super(MATCHUE, self).__init__()

            self.image = image

            self.qepoch = tf.Variable(0.) # _e_      
            self.qepoch.assign_add(tf.cast(1, tf.float32))
            print("qepoch: %s" %(self.qepoch.numpy()))

    matchue = MATCHUE()

    # import pathlib
    # dstpath = os.path.join(gwork,'flower_photos.tar.gz')
    # data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    #                                    fname=dstpath)
    # data_root = pathlib.Path(data_root_orig)
    # print(data_root, type(data_root), data_root.is_dir())


    # #Hyperparameters and statistics
    # batch_size = 64
    # iterations = 30000

    # d_loss = []
    # g_loss = []

    # # # download pokemon.npy
    # # !gdown https://drive.google.com/uc?id=1cmu3uuBH91E_laIC7wnCellfJv5cSa0O


    # #Import data
    # x_train = np.load('data.npy')


    # #Normalize data
    # x_train = x_train.astype('float32') / 255.0

    # image_indices = np.random.randint(0, x_train.shape[0] - 1, [4])
    # real_images = x_train[image_indices]

    # for i in range(4):
    #     plt.figure(i)
    #     plt.imshow(real_images[i])

    # plt.show()

In [0]:
ls -la "/content/gdrive/My Drive/gwork/flower_photos"

# === MNIST

In [0]:
# MODEL MNIST

import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K


class MNISTMODEL(tf.keras.models.Model):

    def __init__(self, 
               img_rows=128, 
               img_cols=128, 
               channels=4, # 
               latent_dim=3, # !
               loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(1e-4)
               ):
    
        super(MNISTMODEL, self).__init__()

        self.qepoch = tf.Variable(0.) # _e_

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = latent_dim

        self.loss = loss
        self.optimizer = optimizer

        self.generator = self.make_generator_model()
        self.discriminator = self.make_discriminator_model()

        # will train two networks separately
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

        # method returning a helper function to compute cross entropy loss
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

        # Loss function for training:
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

        # Train loss
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')
        self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        # Test loss
        self.test_loss = tf.keras.metrics.Mean(name='test_loss')
        self.test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

        self.chkpt = {
            'generator_optimizer': self.generator_optimizer,
            'discriminator_optimizer': self.discriminator_optimizer,
            'generator': self.generator,
            'discriminator': self.discriminator,
            'qepoch': self.qepoch,
        }


    # compare the discriminator's predictions on real images to an array of 1s, 
    # and the discriminator's predictions on fake (generated) images to an array of 0s
    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss    

    # compare the discriminators decisions on the generated images to an array of 1s
    def generator_loss(self, fake_output):
        return self.cross_entropy(tf.ones_like(fake_output), fake_output)      

    # The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers 
    def make_generator_model(self):

        # # ================================ mnist
        model = tf.keras.Sequential()

        model.add(tf.keras.layers.Dense(7*7*256, 
                                use_bias=False, 
                                input_shape=(100,)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Reshape((7, 7, 256)))
        assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

        model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), 
                                        padding='same', use_bias=False))
        assert model.output_shape == (None, 7, 7, 128)
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())

        model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), 
                                        padding='same', use_bias=False))
        assert model.output_shape == (None, 14, 14, 64)
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())

        model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', 
                                        use_bias=False, activation='tanh'))
        
        img_rows = 28
        img_cols = 28
        channels = 1
        assert model.output_shape == (None, img_rows, img_cols, channels)

        return model
 


    # The discriminator is a CNN-based image classifier.
    def make_discriminator_model(self):
        img_rows = 28
        img_cols = 28
        channels = 1

        # ================================ mnist    
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                        input_shape=[img_rows, img_cols, channels]))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(0.3))

        model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(0.3))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1))

        return model


    # *************
    #   call
    def call(self, inputs):
        G = self.generator
  
        generated_image = G(inputs, training=False)
        return generated_image

    # *************
    #   train
    def train(self, 
              inputs,
              epochs_batch = 5,
              batch_size = 256,
              buffer_size = 60000,
              imgtracefolder = 'imgtrace',
              ckptfolder = 'mnistckpt', # gwork + key for ckpoints
              noise_dim = 100, # is latent_dim
              epochs = 50, # 
              num_examples_to_generate = 16 #
              ):   
      
        model = self

        self.epochs_batch = epochs_batch
        self.batch_size = batch_size
        self.imgtracefolder = imgtracefolder
        self.ckptfolder = ckptfolder

        self.noise_dim = noise_dim
        self.num_examples_to_generate = num_examples_to_generate

        # to visualize progress in the animated GIF)
        self.seed = tf.random.normal([self.num_examples_to_generate, 
                                      self.noise_dim])

        self.train_loop(inputs, epochs)

    # ******************
    #   train_step
    #       
    #   Notice the use of `tf.function`
    #   This annotation causes the function to be "compiled"
    #
    @tf.function
    def train_step(self, images):
        model = self
        batch_size = self.batch_size
        noise_dim = self.noise_dim

        noise = tf.random.normal([batch_size, noise_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = model.generator(noise, training=True)

            real_output = model.discriminator(images, training=True)
            fake_output = model.discriminator(generated_images, training=True)

            gen_loss = model.generator_loss(fake_output)
            disc_loss = model.discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, 
                                                model.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                        model.discriminator.trainable_variables)

        model.generator_optimizer.apply_gradients(zip(gradients_of_generator, 
                                                model.generator.trainable_variables))
        model.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, 
                                                model.discriminator.trainable_variables))    

    # ******************
    #   train_loop
    #       
    #
    def train_loop(self, 
                   dataset, 
                   epochs):

        model = self

        ckpt_prefix = exchecker.get_ckpt_prefix()
        ckpt = exchecker.get_checkpoint() # get model from self
        mgr = exchecker.get_manager(ckpt)
        ckptloadstatus = ckpt.restore(mgr.latest_checkpoint)

        print("ckpt type %s" %type(ckpt))
        print("mgr type %s" %type(mgr))
        print("ckptloadstatus %s" %ckptloadstatus)

        if mgr.latest_checkpoint:
            print("Restored from {}".format(mgr.latest_checkpoint))
        else:
            print("Initializing from scratch.")


        for epoch in range(epochs):
            ckpt.qepoch.assign_add(tf.cast(1, tf.float32))
            tf.py_function(mgr.save, [], [tf.string])
            print("epoch in run: %s; epoch in model: %s" %(epoch, model.qepoch.numpy()))

            start = time.time()

            # ------------------------------
            for i,image_batch in enumerate(dataset):
                startbatch = time.time()

                self.train_step(image_batch) # train_step

                timedeltabatch = time.time() - startbatch
            # ------------------------------

            # Produce images for the GIF as we go
            # display.clear_output(wait=True)
            if (epoch + 1) % self.epochs_batch == 0:
                self.generate_and_save_images(epoch + 1,
                                    self.seed,
                                    self.imgtracefolder)

            # Save the model every nepochs
            if (epoch + 1) % self.epochs_batch == 0:
                ckpt.save()

            print ('Time for epoch {} is {} sec'.format(epoch + 1, 
                                                        time.time()-start))

            rvnepoch = model.qepoch
            tf.dtypes.cast(rvnepoch, tf.int32)

            inepoch = rvnepoch.numpy()
            inepoch = inepoch.astype(int)

            print("rvnepoch type %s" %type(inepoch))
            print("rvnepoch value %s" %inepoch)

        # Generate after the final epoch
        display.clear_output(wait=True)
        self.generate_and_save_images(inepoch,
                                          self.seed,
                                          self.imgtracefolder)        

    # Generate and save image after final epoch

    def generate_and_save_images(self,
                                 epoch, 
                                 test_input,
                                 imgtracefolder):

        model = self 

        # Notice `training` is set to False.
        # This is so all layers run in inference mode (batchnorm).
        predictions = model(test_input, training=False)

        fig = plt.figure(figsize=(4,4))

        if (imgtracefolder):
            path = imgtracefolder + '/' + 'image_at_epoch_{:04d}.png'.format(epoch)
        else:
            path = 'image_at_epoch_{:04d}.png'.format(epoch)

        print("generate_and_save_images: show")
        for i in range(predictions.shape[0]):
            plt.subplot(4, 4, i+1)
            plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')

        plt.savefig(path)
        plt.show()


#   ***********************
#
#   TRAIN
#

# DATA
from IPython import display

if 1 or MODITEM == 'MINST':

    dstpath='mnist.npz'
    urd = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/'
    urf = 'mnist.npz'

    datagen = DATAGEN()
    data = datagen.load_data(dstpath, urd, urf)
    print("data type: %s, len %s\n" %(type(data), len(list(data))))

    (train_images, train_labels), (_, _) = data
    print("train_images type: %s, len %s\n" %(type(train_images), len(list(train_images))))

    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
    train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

    # DATA
    buffer_size = 60000
    batch_size = 256


    # Batch and shuffle the data
    train_ds = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)


    # sliced_images =  tf.data.Dataset.from_tensor_slices(train_images)
    # shuffled_images = sliced_images.shuffle(buffer_size)
    # print("shuffled_images type: %s, len %s\n" %(type(shuffled_images), len(list(shuffled_images))))


    # batch_of_images = shuffled_images.batch(batch_size) # !!!
    # print("batch_of_images type: %s, len %s\n" %(type(batch_of_images), len(list(batch_of_images))))

    # train_ds = shuffled_images # 
    print("train_ds type: %s, len %s\n" %(type(train_ds), len(list(train_ds))))

# TRAIN
if 1:

    import matplotlib.pyplot as plt

    # ds = iter(train_dataset) # batch.shape: (256, 28, 28, 1)
    # print("ds type: %s\n" %type(ds))

    # batch = next(ds) # tf.Tensor (256, 28, 28, 1)
    # elem = batch[0]
    # shape = elem.shape 
    # print("elem shape: %s\n" %shape)

    # fig, ax = plt.subplots(figsize=(18, 2))
    # ax.imshow(elem[:, :, 0], cmap='gray')

    # MNISTMODEL
    noise_dim = 100
    mnistmodel = MNISTMODEL(
        noise_dim
        )

    # EXCHECKER
    exchecker = EXCHECKER(mnistmodel,
                        checkpoint_dir = gwork + '/mnistckpts')    
    checkpoint = exchecker.get_checkpoint() # <tensorflow.python.training.tracking.util.Checkpoint object at 0x7f3bf346f588>

    # # PRE-TRAIN
    G = mnistmodel.generator
    D = mnistmodel.discriminator


    noise = tf.random.normal([1, noise_dim])
    generated_image = G(noise, training=False)

    print ("show generated image from noice")
    plt.imshow(generated_image[0, :, :, 0], cmap='gray')
    decision = D(generated_image)
    print ("is real: %s\n" %decision)


    # TRAINER
    tracefolder = gwork + '/mnistimgtrace'
    if not os.path.exists(tracefolder):
        os.mkdir(tracefolder)

    mnistmodel.train(train_ds,
                     epochs = 50, # will run epochs
                     epochs_batch = 5, # checkout by
                     batch_size = 256, # with batch of
                     imgtracefolder = tracefolder)


    # ANIME

    anime = ANIME(model)

    gif = anime.generate_gif(srcfolder=gwork + '/mnistimgtrace', 
                             dstfile = 'ani.gif')         
    anime.display_gif(giffile='ani.gif')

# === STYLEBASE

In [0]:
#   STYLEBASE
#
class STYLEBASE(tf.keras.models.Model):

    def __init__(self,
                 image,
                 verbose = True,
                 ):

        super(STYLEBASE, self).__init__()

        self.image = image


    def predict(self):
        #
        #   Load a VGG19 with the classification head (include_top=True)
        #   and test run it on the content image to ensure it's used correctly
        #
        x = tf.keras.applications.vgg19.preprocess_input(self.image*255)
        x = tf.image.resize(x, (224, 224))
        vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

        prediction_probabilities = vgg(x)
        prediction_probabilities.shape

        predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
        return [(class_name, prob) for (number, class_name, prob) in predicted_top_5]

#   ***********************
#
#   TRAIN
#

if 0:
    # DATA STYLER

    from shutil import copyfile

    Datagen = DATAGEN()
    #
    # to style - nasa
    #
    tostylefolder = "nasa/"
    gtostylefolder = gdata + tostylefolder
    print("gtostylefolder: %s" %gtostylefolder)

    URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
    URFS_A = [
            "potw1934a.jpg", 
            ]
    DIR_A = tostylefolder


    Datagen.get_rawdata(DIR_A, URD_A, URFS_A )

    # if folder of images to style does not e xist
    if not os.path.exists(tostylefolder):
        os.mkdir(tostylefolder)

    # if gdata folder is available
    if os.path.exists(gtostylefolder):
        Datagen.get_gdata(gtostylefolder, tostylefolder)

    # if gdata folder not available, download data
    else:
        Datagen.get_rawdata(DIR_A, URD_A, URFS_A)



    styledfolder = "cortes/"
    gstyledfolder = gdata + styledfolder
    print("gstyledfolder: %s" %gstyledfolder)

    URD_B = 'https://rehs.com/catalogimages/'
    URFS_B = [
            'edouard_cortes_e1339_pont_neuf_wm.jpg',
            ]
    DIR_B = styledfolder

    # if folder of images to style does not e xist
    if not os.path.exists(styledfolder):
        os.mkdir(styledfolder)

    # if gdata folder is available
    if os.path.exists(gstyledfolder):
        Datagen.get_gdata(gstyledfolder, styledfolder)

    # if gdata folder not available, download data
    else:
        Datagen.get_rawdata(DIR_B, URD_B, URFS_B) # tf.keras.utils.get_file(file, url)


    # SINGLE IMAGEs

    Imagine = IMAGINE()

    content_path = os.path.join(DIR_A, URFS_A[0])
    content_image = Imagine.load_img(content_path)

    style_path = os.path.join(DIR_B, URFS_B[0])
    style_image = Imagine.load_img(style_path)


    # content_image
    stylebase = STYLEBASE(content_image)
    prediction = stylebase.predict()
    print(prediction)

# === STYLEHUB


In [0]:
#
#   Fast Style Transfer using TF-Hub
#
#
#   STYLEHUB
#
import tensorflow_hub as hub

class STYLEHUB():

    def __init__(self,
                 style_image,
                 content_image,
                 verbose = True,
                 ):
        
        super(STYLEHUB, self).__init__()

        self.style_image = style_image
        self.content_image = content_image
        self.verbose = verbose

    def predict(self):
        content_image = self.content_image
        style_image = self.style_image

        hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
        stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]

        return stylized_image


#   ******************
#
#   DATA
#
if 0:
    from shutil import copyfile

    Datagen = DATAGEN()
    #
    # to style - nasa
    #
    tostylefolder = "nasa/"
    gtostylefolder = gdata + tostylefolder
    print("gtostylefolder: %s" %gtostylefolder)

    URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
    URFS_A = [
            "potw1934a.jpg", 
            ]
    DIR_A = tostylefolder


    Datagen.get_rawdata(DIR_A, URD_A, URFS_A )

    # if folder of images to style does not e xist
    if not os.path.exists(tostylefolder):
        os.mkdir(tostylefolder)

    # if gdata folder is available
    if os.path.exists(gtostylefolder):
        Datagen.get_gdata(gtostylefolder, tostylefolder)

    # if gdata folder not available, download data
    else:
        Datagen.get_rawdata(DIR_A, URD_A, URFS_A)



    styledfolder = "cortes/"
    gstyledfolder = gdata + styledfolder
    print("gstyledfolder: %s" %gstyledfolder)

    URD_B = 'https://rehs.com/catalogimages/'
    URFS_B = [
            'edouard_cortes_e1339_pont_neuf_wm.jpg',
            ]
    DIR_B = styledfolder

    # if folder of images to style does not e xist
    if not os.path.exists(styledfolder):
        os.mkdir(styledfolder)

    # if gdata folder is available
    if os.path.exists(gstyledfolder):
        Datagen.get_gdata(gstyledfolder, styledfolder)

    # if gdata folder not available, download data
    else:
        Datagen.get_rawdata(DIR_B, URD_B, URFS_B) # tf.keras.utils.get_file(file, url)


    # SINGLE IMAGEs
    content_path = os.path.join(DIR_A, URFS_A[0])
    content_image = Imagine.load_img(content_path)

    style_path = os.path.join(DIR_B, URFS_B[0])
    style_image = Imagine.load_img(style_path)


    # EVAL

    stylehub = STYLEHUB(style_image, content_image)       
    image = stylehub.predict()

    plt.subplot(1, 2, 1)
    Imagine.imshow(image, 'image')


# === MODELVGG


In [0]:
#   MODELVGG
#
class MODELVGG(tf.keras.models.Model):

    def __init__(self,
                 image,
                 verbose = True,
                 ):

        super(MODELVGG, self).__init__()
        self.image = image

    def predict(self):
        #
        #   Load a VGG19 with the classification head (include_top=True)
        #   and test run it on the content image to ensure it's used correctly
        #
        x = tf.keras.applications.vgg19.preprocess_input(self.image*255)
        x = tf.image.resize(x, (224, 224))
        vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

        prediction_probabilities = vgg(x)
        prediction_probabilities.shape

        predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
        return [(class_name, prob) for (number, class_name, prob) in predicted_top_5]

#   ***********************
#
#   EVAL
#

if 0:
    # DATA STYLER

    from shutil import copyfile

    datagen = DATAGEN()

    #
    # to style - nasa
    #
    gdir_a = gdata + '/nasa'
    wdir_a = gwork + '/nasa'
    print("gdir_a: %s" %gdir_a)
    print("wdir_a: %s" %wdir_a)

    URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
    URFS_A = [
            "potw1934a.jpg", 
            ]

    datagen.get_uriitems(URD_A, URFS_A, gdir_a, wdir_a)

    # 
    #   styled =cortes
    #
    gdir_b = gdata + '/cortes'
    wdir_b = gwork + '/cortes'
    print("gdir_b: %s" %gdir_b)
    print("wdir_b: %s" %wdir_b)

    URD_B = 'https://rehs.com/catalogimages/'
    URFS_B = [
            'edouard_cortes_e1339_pont_neuf_wm.jpg',
            ]

    datagen.get_uriitems(URD_B, URFS_B, gdir_b, wdir_b)


    # SINGLE IMAGEs

    Imagine = IMAGINE()

    content_path = os.path.join(wdir_a, URFS_A[0])
    print("content_path: %s" %content_path)
    content_image = Imagine.load_img(content_path)

    style_path = os.path.join(wdir_b, URFS_B[0])
    print("style_path: %s" %style_path)
    style_image = Imagine.load_img(style_path)

    # content_image
    print("content_image type: %s" %type(content_image))
    modelvgg = MODELVGG(content_image)
    prediction = modelvgg.predict()
    print(prediction)

# === STYLER

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/style_transfer.ipynb

https://github.com/manicman1999/Style-Transfer-TF2.0

In [0]:
#   ***********************
#
#   STYLER
#
#   Build a model that returns the style and content tensors
#
import IPython.display as display

class STYLERMOD(tf.keras.models.Model):

    def __init__(self, 
                 style_image = None,
                 content_image = None,
                 verbose = False,
                 total_variation = True,
                 ):

        super(STYLERMOD, self).__init__()

        self.total_variation=total_variation


        # weight for the total_variation_loss
        self.total_variation_weight=30

        self.style_image = style_image
        self.content_image = content_image

        # Choose intermediate layers from the network to represent the style and content of the image
        # VGG19 Content layer - encoder
        self.content_layers = ['block5_conv2'] 

        # VGG19 Style layer
        self.style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

        self.num_style_layers = len(self.style_layers)
        self.num_content_layers = len(self.content_layers)

        # create an optimizer
        self.opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

        #
        #   model
        #
        self.vgg = self.vgg_layers(self.style_layers + self.content_layers)
        self.vgg.trainable = False

        if verbose:
            for layer in self.vgg.layers:
                print(layer.name)

        self.style_extractor = self.vgg_layers(self.style_layers)
        self.style_outputs = self.style_extractor(style_image*255)
        #Look at the statistics of each layer's output
        if verbose:
            for name, output in zip(self.style_layers, self.style_outputs):
                print(name)
                print("  shape: ", output.numpy().shape)
                print("  min: ", output.numpy().min())
                print("  max: ", output.numpy().max())
                print("  mean: ", output.numpy().mean())
                print()


        #
        #   chkpt
        #
        self.epoch = tf.Variable(0.) # _e_        
        self.chkpt = {
            'vgg': self.vgg,
            'epoch': self.epoch,
        }

    

        #
        #   called on an image, the model returns the gram matrix (style) 
        #   of the style_layers and content of the content_layers
        #
        self.results = self(tf.constant(self.content_image)) # content_image
        self.style_results = self.results['style']

        if verbose:
            print('Styles:')
            for name, output in sorted(self.results['style'].items()):
                print("  ", name)
                print("    shape: ", output.numpy().shape)
                print("    min: ", output.numpy().min())
                print("    max: ", output.numpy().max())
                print("    mean: ", output.numpy().mean())
                print()

            print("Contents:")
            for name, output in sorted(self.results['content'].items()):
                print("  ", name)
                print("    shape: ", output.numpy().shape)
                print("    min: ", output.numpy().min())
                print("    max: ", output.numpy().max())
                print("    mean: ", output.numpy().mean())


    #   *******************
    #   vgg_layers
    #
    def vgg_layers(self, layer_names):

        # load a VGG19 without the classification head

        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        print()
        for layer in vgg.layers:
           continue # print(layer.name)

        """ Creates a vgg model that returns a list of intermediate output values."""
        # Load our model. Load pretrained VGG, trained on imagenet data
        
        inputs = [vgg.input]
        outputs = [vgg.get_layer(name).output for name in layer_names]

        model = tf.keras.Model(inputs, outputs)
        return model
        
    #   ***************
    #   style_content_loss
    #
    def style_content_loss(self, outputs):

        style_image = self.style_image
        content_image = self.content_image

        # weighted combination of the two losses to get the total loss
        style_weight=1e-2
        content_weight=1e4

        # set style and content target values
        style_targets = self(style_image)['style']
        content_targets = self(content_image)['content']

        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                            for name in style_outputs.keys()])
        style_loss *= style_weight / self.num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                                for name in content_outputs.keys()])
        content_loss *= content_weight / self.num_content_layers
        loss = style_loss + content_loss
        return loss

    # *************
    #   gram_matrix
    #   The content of an image is represented by the values of the intermediate feature maps
    #   the style of an image can be described by the means and correlations across the different feature map
    #
    #   Calculate a Gram matrix that includes the style by taking the outer product 
    #   of the feature vector with itself at each location, and averaging that outer 
    #   product over all locations 
    #
    def gram_matrix(self, input_tensor):
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)

    # *************
    #   call
    #   When called on an image, this model returns the gram matrix (style) 
    #   of the style_layers and content of the content_layers
    #
    #   return the style and content tensors
    #
    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        
        # preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        preprocessed_input = inputs

        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                        outputs[self.num_style_layers:])

        style_outputs = [self.gram_matrix(style_output)
                        for style_output in style_outputs]

        content_dict = {content_name:value 
                        for content_name, value 
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name:value
                    for style_name, value
                    in zip(self.style_layers, style_outputs)}
        
        return {'content':content_dict, 'style':style_dict}


    # *************
    #   train
    #
    def train(self, 
              inputs,
                 outimgfolder = 'outimgfolder',
                 imgtracefolder = 'imgtrace',
                 ckptfolder = 'mnistckpt',
                 batch_size = 128,
                 epochs = 50,
                 epochs_batch = 10,
                 noise_dim = 3,
                 save_interval = 100,
                 latent_dim = 128,
                 total_variation = True,
                 num_examples_to_generate = 16,
                 steps_per_epoch = 100
              ):  
    
        model = self

        self.train_loop(inputs, 
                        epochs, 
                        steps_per_epoch, 
                        epochs_batch,
                        imgtracefolder,
                        total_variation,
                        )

    #   **********************
    #   total_variation_loss
    #
    # Use tf.GradientTape to update the image
    def total_variation_loss(self, image):
        x_deltas, y_deltas = high_pass_x_y(image)
        return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

    #   **********************
    #   train_step
    #
    @tf.function()
    def train_step(self, image, total_variation = True):
        
        model = self

        with tf.GradientTape() as tape:
            outputs = model(image)
            loss = model.style_content_loss(outputs)

            if total_variation:
                loss += self.total_variation_weight*tf.image.total_variation(image)

        grad = tape.gradient(loss, image)
        model.opt.apply_gradients([(grad, image)])
        image.assign(Imagine.clip_0_1(image))

    #   **********************
    #   train_loop
    #
    def train_loop(self, 
                   inputs, 
                   epochs = 10, 
                   epochs_batch = 10,
                   steps_per_epoch = 80,
                   imgtracefolder = "imgtrace",
                   total_vatiation = True):
      
        model = self
        image = inputs

        print("train_loop epochs: %s , steps: %s " %(epochs, steps_per_epoch))

        ckpt_prefix = exchecker.get_ckpt_prefix()
        ckpt = exchecker.get_checkpoint() # get model from self
        mgr = exchecker.get_manager(ckpt)
        ckptloadstatus = ckpt.restore(mgr.latest_checkpoint)

        print("ckpt type %s" %type(ckpt))
        print("mgr type %s" %type(mgr))
        print("ckptloadstatus %s" %ckptloadstatus)

        if mgr.latest_checkpoint:
            print("Restored from {}".format(mgr.latest_checkpoint))
        else:
            print("Initializing from scratch.")


        step = 0
        for epoch in range(epochs):
            ckpt.epoch.assign_add(tf.cast(1, tf.float32))
            tf.py_function(mgr.save, [], [tf.string])
            print("epoch in run: %s; epoch in model: %s" %(epoch, model.epoch.numpy()))

            start = time.time()

            # ------------------------------
            for m in range(steps_per_epoch):
                step += 1
                tim = time.time()
                # if (epoch + 1) % epochs_batch == 0:
                #     print('train epoch %s step %s at %s' %(epoch, m, tim))
                if 0:   # would display image
                    display.display(Imagine.tensor_to_image(image))

                # --------------------------

                self.train_step(image, total_variation = True)

                # --------------------------
                print(".", end='')
            # ------------------------------        

            # Save the model every nepochs
            
            if (epoch + 1) % epochs_batch == 0:
                ckpt.save()

            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

            rvnepoch = model.epoch
            tf.dtypes.cast(rvnepoch, tf.int32)

            inepoch = rvnepoch.numpy()
            inepoch = inepoch.astype(int)

            print("rvnepoch %s %inepoch type %s" %type(inepoch))
            
            if (epoch + 1) % epochs_batch == 0:
                self.generate_and_save_images(image, inepoch + 1, imgtracefolder)

        # Generate after the final epoch
        display.clear_output(wait=True)

        self.generate_and_save_images(image, epoch + 1, imgtracefolder)

    # Generate and save images

    def generate_and_save_images(self, 
                                 image, 
                                 epoch,
                                 imgtracefolder):
        
        img = Imagine.tensor_to_image(image)

        if (imgtracefolder):
            if not os.path.exists(imgtracefolder):        
                os.mkdir(imgtracefolder)              
            path = imgtracefolder + '/' + 'image_at_epoch_{:04d}.png'.format(epoch)
        else:
            path = 'image_at_epoch_{:04d}.png'.format(epoch)

        display.clear_output(wait=True)

        plt.savefig(path)
        # display.display(img)
        plt.show()

#   ************************
#
#   DO
#
if 1:

    import matplotlib.pyplot as plt
    from PIL import Image

    from shutil import copyfile

    Imagine = IMAGINE()

    datagen = DATAGEN()

    #
    # to style - nasa
    #
    gdir_a = gdata + '/nasa'
    wdir_a = gwork + '/nasa'
    # print("gdir_a: %s" %gdir_a)
    # print("wdir_a: %s" %wdir_a)

    URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
    URFS_A = [
            "potw1934a.jpg", 
            "potw1935a.jpg", 
            "potw1936a.jpg",
            "potw1937a.jpg",
            "potw1938a.jpg",
            "potw1939a.jpg",
            "potw1940a.jpg",
            "potw1941a.jpg",
            "potw1942a.jpg",
            "potw1943a.jpg",
            ]
    datagen.get_uriitems(URD_A, URFS_A, gdir_a, wdir_a)


    # 
    #   styled =cortes
    #
    gdir_b = gdata + '/cortes'
    wdir_b = gwork + '/cortes'
    # print("gdir_b: %s" %gdir_b)
    # print("wdir_b: %s" %wdir_b)

    URD_B = 'https://rehs.com/catalogimages/'
    URFS_B = [
            'edouard_cortes_e1339_pont_neuf_wm.jpg',
            'edouard_leon_cortes_e1312_bouquinistes_wm.jpg',
            'edouard_leon_cortes_e1114_moulin_de_la_galette_soir_de_neige_wm.jpg',
            'edouard_leon_cortes_e1148_boulevard_des_allies_church_of_saint_pierre_caen_wm.jpg',
            ]
    datagen.get_uriitems(URD_B, URFS_B, gdir_b, wdir_b)

    #   ******************
    #   single images
    #
    content_path = os.path.join(wdir_a, URFS_A[0])
    content_image = Imagine.load_img(content_path)

    style_path = os.path.join(wdir_b, URFS_B[0])
    style_image = Imagine.load_img(style_path)


    #
    #   DATASETs
    #
    XSIZE=128
    YSIZE=128
    NSLICES=10

    BUFFER_SIZE = 6000
    BATCH_SIZE = 256
    CHANNELS = 3

    RESIZE=0.75
    DIRECTORY='nasa/' # 'FluidArt/'
    NAME= 'nasa_256_128' # name='fluid_256_128'

    train_dataset_a = datagen.get_dataset(
                                    path= gwork + '/cortes/',
                                    batch_size = BATCH_SIZE,
                                    buffer_size = BUFFER_SIZE,
                                    xSize=XSIZE,
                                    ySize=YSIZE,
                                    nSlices=NSLICES,
                                    resize=RESIZE,
                                    channels=CHANNELS
                                    )

    typeds = type(train_dataset_a) # <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'> 
    # print("train_dataset_a type: %s" %(type(train_dataset_a)))

    it1_a = iter(train_dataset_a) # batch.shape: (256, 28, 28, 1)
    batch_a = next(it1_a)
    images_a = np.array(batch_a)
    x_train_a = images_a

    # assert( x_train_a.shape == (nimages * NSLICES, XSIZE, YSIZE, CHANNELS))
    assert(isinstance(x_train_a, np.ndarray))

    if 0:
        fig, axs = plt.subplots(MOSAIC_ROWS, MOSAIC_COLS)
        for i in range(MOSAIC_ROWS):
            for j in range(MOSAIC_COLS):
                axs[i,j].imshow( x_train_a[ np.random.randint(x_train_a.shape[0]) ] )
                axs[i,j].axis('off')
        plt.show()

    train_dataset_b = datagen.get_dataset(
                                    path= gwork + '/nasa/',
                                    batch_size = BATCH_SIZE,
                                    buffer_size = BUFFER_SIZE,
                                    xSize=XSIZE,
                                    ySize=YSIZE,
                                    nSlices=NSLICES,
                                    resize=RESIZE,
                                    channels=CHANNELS)

    typeds = type(train_dataset_b) # <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'> 
    # print("train_dataset_b type: %s" %(type(train_dataset_b)))

    it1_b = iter(train_dataset_b) # batch.shape: (256, 28, 28, 1)
    batch_b = next(it1_b)
    images_b = np.array(batch_b)
    x_train_b = images_b

    # assert( x_train_b.shape == (nimages * NSLICES, XSIZE, YSIZE, CHANNELS))
    assert(isinstance(x_train_b, np.ndarray))

    #   *****************
    #   SHOW
    #
    if 0:
        MOSAIC_ROWS = 6
        MOSAIC_COLS = 6    
        fig, axs = plt.subplots(MOSAIC_ROWS, MOSAIC_COLS)
        for i in range(MOSAIC_ROWS):
            for j in range(MOSAIC_COLS):
                axs[i,j].imshow( x_train_b[ np.random.randint(x_train_b.shape[0]) ] )
                axs[i,j].axis('off')
        plt.show()


    #   *****************
    #   TRAIN
    #
    content_inputs = tf.Variable(content_image)
    print("style_image to model, content_image to train_step")

    stylermod = STYLERMOD(style_image = style_image, content_image = content_image,)
    
    exchecker = EXCHECKER(stylermod, 
                          checkpoint_dir = gwork + '/stylerckpts', clear = False)
    
    stylermod.train_step(content_inputs)

    stylermod.train_loop(content_inputs, 
                    epochs = 10, 
                    epochs_batch = 1, # checkout by
                    steps_per_epoch = 100,
                    imgtracefolder = gwork + '/stylertrace')

    display.display(Imagine.tensor_to_image(content_inputs))

    



In [0]:
# ANIME

anime = ANIME(stylermod)
gif = anime.generate_gif(srcfolder=gwork + '/stylertrace', dstfile = 'ani.gif')
anime.display_gif(giffile='ani.gif')


# === ART2GAN

In [0]:
## MODEL ART2GAN

from IPython import display
from IPython.display import clear_output

import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

class ART2GAN(Model):
    def __init__(self,
               img_rows=128, 
               img_cols=128, 
               channels=3, # RGB
               latent_dim=256,
               imgpath = 'imgpath', # img holder 
               ckptpath  = 'ckptpath', # checkpoints place
               tracepath  = 'tracepath', # in loop folder
               ):
    
        super(ART2GAN, self).__init__()

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = latent_dim
        print("ART2GAN latent_dim: %s" %latent_dim)

        self.imgpath = imgpath
        self.ckptpath = ckptpath
        self.tracepath = tracepath

        self.loss = 'binary_crossentropy'
        self.optimizer = Adam(0.0005, 0.6)
        self.generator_optimizer = self.optimizer
        self.discriminator_optimizer = self.optimizer
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)  
        self.discriminator = self.build_discriminator()
        self.generator = self.build_generator()
        self.dcgan_optimizer = self.optimizer
        self.build_dcgan()

        self.qepoch = tf.Variable(0.)
        qepoch = self.qepoch.numpy() + 0
        print("************** qepoch : %s" %qepoch )

        self.chkpt = {
            'generator': self.generator,
            'generator_optimizer': self.generator_optimizer,
            'discriminator': self.discriminator,
            'discriminator_optimizer': self.discriminator_optimizer,
            'dcgan': self.dcgan,
            'dcgan_optimizer': self.dcgan_optimizer,
            'qepoch': self.qepoch,
        }

    # ================================ dcgan    

    def build_dcgan(self):
        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the dcgan model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The dcgan model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.dcgan = Model(z, valid)
        self.dcgan.compile(loss=self.loss, optimizer=self.dcgan_optimizer)    
    
    def load_weights(self, 
                     generator_file=None, 
                     discriminator_file=None):
        print("DCGAN.load_weights %s %s" %(generator_file, discriminator_file))
        if generator_file:
            print("load generator weights %s" %s)
            generator = self.build_generator()
            generator.load_weights(generator_file)
            self.generator = generator
        else:
            print("generator weights not found: %s" %s)
    
        if discriminator_file:
            print("load discriminator_file weights %s" %s)
            discriminator = self.build_discriminator()
            discriminator.load_weights(discriminator_file)
            self.discriminator = discriminator
        else:
            print("discriminator_file weights not found: %s" %s)

        if generator_file or discriminator_file: 
            self.build_dcgan() 
            
    # ================================ generator

    # The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers 
    def build_generator(self):

        model = tf.keras.Sequential()
        
        #model.add(Dense(128, activation="relu", input_dim=self.latent_dim, name="generator_input") )
        #model.add(Dropout(0.1))
                
        model.add(layers.Dense(64 * 32 * 32, activation="relu",
                               input_dim=self.latent_dim, 
                               name="generator_input") )
        model.add(tf.keras.layers.Dropout(0.3))
        model.add(tf.keras.layers.Reshape((32, 32, 64)))

        model.add(layers.Conv2D(64, kernel_size=4, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.Activation("relu"))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.UpSampling2D())
        
        model.add(layers.Conv2D(32, kernel_size=4, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.Activation("relu"))
        model.add(tf.keras.layers.UpSampling2D())

        model.add(layers.Conv2D(32, kernel_size=4, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(tf.keras.layers.Activation("relu"))

        model.add(Conv2D(self.channels, kernel_size=3, padding="same", 
                        activation="sigmoid", name="generator_output"))


        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img, name="generator")   

    # ================================ discriminator

    # The discriminator is a CNN-based image classifier.
    def build_discriminator(self):

        model = Sequential()

        model.add(layers.Conv2D(32, kernel_size=3, strides=2, 
                                input_shape=self.img_shape, padding="same"))
        model.add(layers.LeakyReLU(alpha=0.2))

        model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(tf.keras.layers.ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.LeakyReLU(alpha=0.2))

        model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.Dropout(0.25))

        model.add(layers.Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.Dropout(0.25))
        model.add(Flatten())

        #model.add(Dense(32, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        # model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        discrim = Model(img, validity)

        discrim.compile(loss='binary_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy'])

        return discrim

    #   *************
    #   train_loop
    #
    def train_loop(self, 
                     X_train, 
                     epochs, 
                     batch_size=128, 
                     epochs_batch = 10,
                     save_interval=100):

        model = self

        ckpt = exchecker.get_checkpoint() # get model from self
        mgr = exchecker.get_manager(ckpt)
        ckptloadstatus = ckpt.restore(mgr.latest_checkpoint)

        print("ckpt type %s" %type(ckpt))
        print("mgr type %s" %type(mgr))
        print("ckptloadstatus %s" %ckptloadstatus)

        if mgr.latest_checkpoint:
            print("Restored from {}".format(mgr.latest_checkpoint))
        else:
            print("Initializing from scratch.")

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            
            ckpt.qepoch.assign_add(tf.cast(1, tf.float32))
            tf.py_function(mgr.save, [], [tf.string])
            qepoch = int(model.qepoch.numpy())
            print("epoch in run: %s; epoch in model: %s" %(epoch, qepoch))

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.dcgan.train_on_batch(noise, valid)

            # Plot the progress
            if epoch % save_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                print("save trace image")
                self.save_imgs( "img_{:05d}.png".format(qepoch) ) # pass just name
  

    #   *************
    #   __train_loop
    #
    def __train_loop(self,
                   images,
                   epochs = 50,
                   batch_size = 32,
                   save_interval = 100,
                   epochs_batch = 10,
                   num_examples_to_generate = 16):
        
        print("train_loop - epochs %s, batch_size: %s" %(epochs, batch_size))

        ckpt_prefix = Exchecker.get_ckpt_prefix()
        ckpt = Exchecker.get_checkpoint()
        mgr = Exchecker.get_manager(ckpt)
        print("train loop get mgr")
        ckptloadstatus = ckpt.restore(mgr.latest_checkpoint)
        print("ckptloadstatus %s" %ckptloadstatus)

        if mgr.latest_checkpoint:
            print("Restored from {}".format(mgr.latest_checkpoint))
            self.latestchkpt = mgr.latest_checkpoint
        else:
            print("Initializing from scratch.")

        num_examples_to_generate = 16
        # to visualize progress in the animated GIF)
        self.seed = tf.random.normal([num_examples_to_generate, self.latent_dim])

        for epoch in range(epochs):

            ckpt.qepoch.assign_add(tf.cast(1, tf.float32))
            tf.py_function(mgr.save, [], [tf.string])
            print("epoch in run: %s; qepoch in model: %s" %(epoch, self.qepoch + 0))

            start = time.time()

            # =========================

            self.train_step(images, epoch, batch_size, save_interval, epochs_batch)

            # =========================

            # Save the model every nepochs
            if (epoch + 1) % epochs_batch == 0:
                ckpt.save()

            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

            rvnepoch = self.qepoch
            tf.dtypes.cast(rvnepoch, tf.int32) 

            nepoch = rvnepoch.numpy()
            nepoch = nepoch.astype(int)
            print("rvnepoch rvnepoch value %s type %s " %(rvnepoch, type(rvnepoch)))

            if (epoch + 1) % save_interval == 0:
                display.clear_output(wait=True)
                self.generate_and_save_images(nepoch, self.seed)        
    
    #   **************
    #   train_step
    #
    #   Notice the annotation `tf.function`
    #   This annotation causes the function to be "compiled".
    #
    @tf.function
    def train_step(self, 
                   X_train, 
                   epoch, 
                   batch_size, 
                   save_interval,
                   epochs_batch,
                   verbose = False):

        if verbose:
            print("train_step - X_train type: %s" %type(X_train))
            print("train_step - img shape: %s" %X_train[0].shape)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        print("noise shape:", noise.shape)

        d_loss_real = self.discriminator.train_on_batch(imgs, valid)
        d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = self.dcgan.train_on_batch(noise, valid)

        # Plot the progress
        if epoch % epochs_batch == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    #   ***********
    #   Generate and save images
    #
    def generate_and_save_images(self,
                                 epoch, 
                                 test_input):

        model = self.generator 
        tracepath = self.tracepath

        print("aa2 generate_and_save_images for epoch %s" %epoch)

        # Notice `training` is set to False.
        # This is so all layers run in inference mode (batchnorm).
        # predictions = model(test_input, training=False)
        predictions = model.predict(test_input)

        fig = plt.figure(figsize=(4,4))

        try:
            gwork
        except NameError:
            gwork = "" # if not exist assume no g prefix

        print("img trace prefix: %s" %tracepath)
        path = tracepath + '/' + 'image_at_epoch_{:04d}.png'.format(epoch)

        for i in range(predictions.shape[0]):
            plt.subplot(4, 4, i+1)
            plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
            plt.axis('off')

        plt.savefig(path)
        plt.show()

    #   ***********
    #   Generate and save images
    #
    def save_imgs(self, name='img.png'):
        print("aa2 save_imgs for name %s" %name)
        tracepath = self.tracepath

        latent_dim = self.latent_dim
        predict = self.generator.predict

        r, c = 4, 4
        noise = np.random.normal(0, 1, (r * c, latent_dim))

        # replace the first two latent variables with known values
        #for i in range(r):
        #    for j in range(c):
        #        noise[4*i+j][0] = i/(r-1)-0.5
        #        noise[4*i+j][1] = j/(c-1)-0.5

        gen_imgs = predict(noise)

        fig, axs = plt.subplots(r, c, figsize=(6.72,6.72))
        plt.subplots_adjust(left=0.05,bottom=0.05,right=0.95,top=0.95,wspace=0.2,hspace=0.2)

        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i,j].axis('off')
                cnt += 1

        fig.savefig(tracepath + '/' + '{}'.format(name) )

        plt.close()
    


In [0]:
#   *****************************
#
#   TRAIN
#
if (1 or MODITEM == 'ART2GAN'):

    CLEARTMP = 0 # clear g tmp folder 

    from PIL import Image
    from shutil import copyfile

    datagen = DATAGEN()
    Imagine = IMAGINE()

    gsrcdir = gdata + "nasag/"
    print("gsrcdir: %s" %gsrcdir)

    URD_A = 'http://images-assets.nasa.gov/image/PIA03096/'
    URFS_A = [ "PIA03096~orig.jpg",  ]

    wdstdir = gwork + "nasag/"             
    rawimgdir = wdstdir

    print("gsrcdir %s " %gsrcdir)
    print("wdstdir %s " %wdstdir)
    
    datagen.get_uriitems(URD_A, URFS_A, gsrcdir, wdstdir)

    x_train, y_train = datagen.create_dataset_np(128, 128, 
                                    nSlices=1000, 
                                    resize=0.1, 
                                    directory=rawimgdir) #  datagen.create_dataset_np
    shape = x_train.shape
    print("shape: ", shape) # (1000, 128, 128, 3)
    print("shape[0]: ", shape[0])
    assert(shape[0]>0)

    # plot results from datagen
    if 1:
        fig, axs = plt.subplots(4, 4)
        for i in range(4):
            for j in range(4):
                axs[i,j].imshow( x_train[ np.random.randint(x_train.shape[0]) ] )
                axs[i,j].axis('off')
        plt.show()
    

In [0]:
if (1 or MODITEM == 'ART2GAN'):
    # instance model

    prefix = gwork
    mname = 'aa2'
    imgdir = 'imgdir' # img holder 
    ckptdir  = 'ckptdir' # checkpoints place
    tracedir  = 'tracedir' # in loop folder   

    modpath = prefix + '/' + mname
    imgpath = modpath + '/' + imgdir
    ckptpath = modpath + '/' + ckptdir
    tracepath = modpath + '/' + tracedir

    if not os.path.exists(modpath ):
        os.mkdir(modpath )     
    if not os.path.exists(imgpath ):
        os.mkdir(imgpath )        
    if not os.path.exists(ckptpath ):
        os.mkdir(ckptpath )        
    if not os.path.exists(tracepath ):
        os.mkdir(tracepath )        

    art2gan = ART2GAN(img_rows = x_train[0].shape[0],
                    img_cols = x_train[0].shape[1],
                    channels = x_train[0].shape[2], 
                    latent_dim = 256,
                    imgpath = imgpath, # img holder 
                    ckptpath  = ckptpath, # checkpoints place
                    tracepath  = tracepath, # in loop folder                    
                    ) # ?

if (1 or MODITEM == 'ART2GAN'):
    exchecker = EXCHECKER(art2gan,
                          clear = False)  
if (1 or MODITEM == 'ART2GAN'):
    # try:
    #     art2gan.load_weights(generator_file="generator ({}).h5".format(art2gan.mname), discriminator_file="discriminator ({}).h5".format(art2gan.name) )
    # except:
    #     pass       


    art2gan.train_loop(x_train, 
                epochs=110, 
                batch_size=32, 
                save_interval=10,
                epochs_batch = 10)


In [0]:
# ANIME

anime = ANIME(art2gan)
gif = anime.generate_gif(srcfolder = tracepath, dstfile = 'ani.gif')         
anime.display_gif(giffile='ani.gif')

# === BIGAN

https://github.com/manicman1999/Keras-BiGAN

## MODEL

In [0]:
from PIL import Image
from math import floor
import numpy as np
import time
from functools import partial
from random import random
import os

from tensorflow.keras.layers import Conv2D, Dense, AveragePooling2D, Activation, Cropping2D, Dropout, BatchNormalization
from tensorflow.keras.layers import Reshape, UpSampling2D, Flatten, Input, add, Lambda, concatenate, LeakyReLU, multiply
from tensorflow.keras.layers import GlobalAveragePooling2D, average
from tensorflow.keras.models import model_from_json, Model
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from PIL import Image
from shutil import copyfile


im_size = 128
latent_size = 64
BATCH_SIZE = 32
cmode = 'RGB'
channels = 3
size_adjusted = False

directory = "Faces256"
suff = 'jpg'

datafolder = "bigan/"
if not os.path.exists(datafolder):
    os.mkdir(datafolder)

resultsfolder = "Results/"
if not os.path.exists(resultsfolder):
    os.mkdir(resultsfolder)

modelsfolder = "Models/"
if not os.path.exists(modelsfolder):
    os.mkdir(modelsfolder)

rawimgdir = datafolder + "images/"
if not os.path.exists(rawimgdir):
    os.mkdir(rawimgdir)


srcpath = datafolder + directory + str(im_size)
print(f"srcpath: {srcpath}")

URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
URFS_A = [ "potw1934a.jpg", "potw1935a.jpg", "potw1936a.jpg", "potw1937a.jpg", "potw1938a.jpg",
        "potw1939a.jpg", "potw1940a.jpg", "potw1941a.jpg", "potw1942a.jpg", "potw1943a.jpg" ]

gdir_a = gdata + '/nasa'
wdir_a = gwork + rawimgdir

datagen = DATAGEN()
datagen.get_uriitems(URD_A, URFS_A, gdir_a, wdir_a)


class BIGAN(object):

    def __init__(self, steps = 1, lr = 0.0001, decay = 0.00001):

        #Models
        self.D = None
        self.E = None
        self.G = None

        self.GE = None
        self.EE = None

        self.DM = None
        self.AM = None



## DATA

In [0]:
if 0:
    from PIL import Image
    from shutil import copyfile


    # datagen = DATAGEN()
    # Imagine = IMAGINE()

    datafolder = "data/"
    if not os.path.exists(datafolder):
        os.mkdir(datafolder)

    resultsfolder = "Results/"
    if not os.path.exists(resultsfolder):
        os.mkdir(resultsfolder)

    modelsfolder = "Models/"
    if not os.path.exists(modelsfolder):
        os.mkdir(modelsfolder)

    rawimgdir = "data/images/"
    if not os.path.exists(rawimgdir):
        os.mkdir(rawimgdir)


    srcdir = "nasag/"
    gsrcdir = gdata + srcdir
    print("gsrcdir: %s" %gsrcdir)


    URD_A = 'https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/'
    URFS_A = [
            "potw1934a.jpg", 
            "potw1935a.jpg", 
            "potw1936a.jpg",
            "potw1937a.jpg",
            "potw1938a.jpg",
            "potw1939a.jpg",
            "potw1940a.jpg",
            "potw1941a.jpg",
            "potw1942a.jpg",
            "potw1943a.jpg",
            "potw1944a.jpg",
            ]
    DIR_A = rawimgdir


    if 1: # if g src eq g raw, download to raw
        dstdir = rawimgdir
        if not os.path.exists(dstdir):
            print("create imgfolder: %s" %dstdir)
            os.mkdir(dstdir)
    
        if os.path.exists(gsrcdir):
            for file in URFS_A:
                srcfile = os.path.join(gsrcdir, file)
                dstfile = os.path.join(dstdir, file)
                if os.path.exists(srcfile) and not os.path.exists(dstfile):
                    print("copy from gdrive: cp %s to %s" %(srcfile, dstfile))
                    copy(srcfile, dstfile)

                else:
                    print("download from uri %s" %file)
                    uri = URD_A + file
                    !wget --no-check-certificate $uri -P  "$dstdir"

        else:
            for img in URFS_A:
                print(img)
                uri = URD_A + img
                !wget --no-check-certificate $uri -P  $dstdir



## TRAIN

In [0]:
if 0:
    model = BiGAN(lr = 0.0001, silent = False)
    model.evaluate(0)

    while model.GAN.steps <= 600000:
        model.train()

## EVAL

In [0]:
if 0:
    from idw import IDW
    from keras.models import model_from_json
    from PIL import Image
    import numpy as np
    import os
    import matplotlib.pyplot as plt
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.cluster import KMeans
    import random
    import time

    directory = "Earth"
    im_size = 128
    suffix = 'jpg'
    model_num = 15

    def loadModel(mname, num):

        file = open("Models/"+mname+".json", 'r')
        json = file.read()
        file.close()

        mod = model_from_json(json)
        mod.load_weights("Models/"+mname+"_"+str(num)+".h5")

        return mod

    class dataGenerator(object):

        def __init__(self, loc, flip = True, suffix = 'png'):
            self.flip = flip
            self.suffix = suffix
            self.files = []
            self.n = 1e10

            print("Importing Images...")

            try:
                os.mkdir("data/" + loc + "-npy-" + str(im_size))
            except:
                self.load_from_npy(loc)
                return

            for dirpath, dirnames, filenames in os.walk("data/" + loc):
                for filename in [f for f in filenames if f.endswith("."+str(self.suffix))]:
                    print('\r' + str(len(self.files)), end = '\r')
                    fname = os.path.join(dirpath, filename)
                    temp = Image.open(fname).convert(cmode)
                    if not size_adjusted:
                        temp = temp.resize((im_size, im_size), Image.BILINEAR)
                    temp = np.array(temp, dtype='uint8')
                    self.files.append(temp)
                    if self.flip:
                        self.files.append(np.flip(temp, 1))

            self.files = np.array(self.files)
            np.save("data/" + loc + "-npy-" + str(im_size) + "/data.npy", self.files)

            self.n = self.files.shape[0]

            print("Found " + str(self.n) + " images in " + loc + ".")

        def load_from_npy(self, loc):

            print("Loading from .npy files.")

            self.files = np.load("data/" + str(loc) + "-npy-" + str(im_size) + "/data.npy")

            self.n = self.files.shape[0]


        def get_batch(self, num):

            idx = np.random.randint(0, self.n, num)
            out = []

            for i in range(num):
                out.append(self.files[idx[i]])

            return np.array(out).astype('float32') / 255.0

    data = dataGenerator(directory, suffix = suffix)
    encoder = loadModel("enc", model_num)
    generator = loadModel("gen", model_num)

